In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/11 19:46:32 WARN Utils: Your hostname, MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.23.121.76 instead (on interface en0)
25/03/11 19:46:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 19:46:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-22 13:18:32|  2020-01-22 13:45:58|                 N|         1|         244|          41|              1|         5.22|       22.0|  0.0|    0.

In [7]:
df_green.registerTempTable('green')

/usr/local/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [8]:
df_green_revenue = spark.sql("""
    SELECT 
        -- Revenue grouping 
        date_trunc('hour', lpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM 
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1, 2
    """)

In [9]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-06 07:00:00| 241|            169.16|             6|
|2020-01-19 19:00:00|  41| 457.2400000000003|            43|
|2020-01-23 23:00:00| 129| 304.1300000000001|            22|
|2020-01-28 15:00:00| 129|            208.23|             9|
|2020-01-02 08:00:00|  74|1187.4199999999987|            96|
|2020-01-06 16:00:00| 129|250.42000000000002|            15|
|2020-01-07 10:00:00| 171|             38.98|             2|
|2020-01-09 15:00:00| 260| 242.6800000000001|            19|
|2020-01-16 20:00:00|  82| 424.1900000000001|            27|
|2020-01-21 16:00:00|  33| 917.3599999999994|            38|
|2020-01-08 17:00:00| 140|            313.81|             8|
|2020-01-22 08:00:00| 244| 285.3999999999999|            14|
|2020-01-30 14:00:00|  74| 738.6799999999996|            49|
|2020-01-03 07:00:00|  7

In [10]:
df_green_revenue.write.mode('overwrite').parquet('data/report/revenue/green/')

25/03/11 19:47:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [11]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [12]:
df_yellow.registerTempTable('yellow')

In [13]:
df_yellow_revenue = spark.sql("""
    SELECT 
        -- Revenue grouping 
        date_trunc('hour', tpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM 
        yellow
    WHERE
        tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1, 2
    """)

In [14]:
df_yellow_revenue.write.mode('overwrite').parquet('data/report/revenue/yellow/')

25/03/11 19:47:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [15]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [16]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [17]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000001|                   57|
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|
|2020-01-01 00:00:00|  56|             99.69|                   3|              18.1|                    2|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|
|2020-01-01 00:00:00|  65|199.49000000000004|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  74|317.09000000000015|                  24| 586.2100000000002|                   47|
|2020-01-01 00:00:00|  88|  

In [18]:
df_join.write.mode('overwrite').parquet('data/report/revenue/join/')

25/03/11 19:47:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [19]:
df_zones = spark.read.parquet('zones/')

In [20]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [21]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000001|                   57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|  10|              NULL|                NULL|             42.41|                    2|        10|   Queens|        Baisley Park|   Boro Zone|
|2020-01-01 00:00:00|  56|             99.69|                   3|              18.1|                    2|        56|   Queens|              Corona|   Boro Zone|
|2020-01-01 00:00:00| 

In [22]:
df_result.drop('LocationID', 'zone').write.mode('overwrite').parquet('tmp/revenue-zones/')

25/03/11 19:48:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/03/11 20:15:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 981639 ms exceeds timeout 120000 ms
25/03/11 20:15:10 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/11 20:15:17 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [35]:
spark.stop()